In [109]:
from ultralytics import YOLO
import cv2
import csv
import time
import pandas as pd

In [110]:
model = YOLO('/Users/rifqinz/Downloads/Kecilin_Test/runs/detect/train/weights/best.pt')
video= '/Users/rifqinz/Downloads/Kecilin_Test/beyblade2.mp4'
cap = cv2.VideoCapture(video)

In [111]:
# Prepare CSV file for battle data
columns = ['Battle Number', 'Duration (s)', 'Winner', 'Cause', 'Fight Count']
battle_df = pd.DataFrame(columns=columns)

In [112]:
#FUNCTION
# Function to check if Beyblade is out of the arena
# Define the arena area (for example, a bounding box)
arena_top_left = (1000, 1000)  # Top-left corner of the arena in pixels
arena_bottom_right = (1000, 1000)  # Bottom-right corner of the arena in pixels
def is_out_of_arena(bbox):
    x_min, y_min, x_max, y_max = bbox
    if x_min < arena_top_left[0] or y_min < arena_top_left[1] or x_max > arena_bottom_right[0] or y_max > arena_bottom_right[1]:
        return True
    return False
def save_to_dataframe_csv(battle_data):
    global battle_df
    new_row = pd.DataFrame([battle_data], columns=columns)
    battle_df = pd.concat([battle_df, new_row], ignore_index=True)
    battle_df.to_csv('beyblade_battle_data_pandas.csv', index=False)

In [113]:
# Initialize states
time_started = False
battle_ongoing = False
number_battle = 0
hand_launcher_detected = False
detected_text = ""  # Variable to store the detected state text
battle_data = []  # List to store battle results
fight_count = 0  # Variable to count the number of "Fight" (3) occurrences in a battle
timer_text=0
winner='none'
hit=0
cause='none'

In [114]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Model inference for object detection
    results = model(frame)  # Assuming 'model' is already defined
    annotated_frame = results[0].plot()
    data = []  # List to store detected class IDs and bounding boxes
    
    # Extract detected class IDs
    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls)
            bbox = box.xyxy[0].cpu().numpy().astype(int)
            data.append(cls_id)

    #-------------------------------------Condition---------------------------------------------#
    # Condition when both Hand (4) and Launcher (5) appear first, set flag
    if 4 in data and 5 in data:
        hand_launcher_detected = True
        print("Hand and Launcher detected, waiting for Beyblade to spin...")

    # Start the timer when both Blue_Beyblade_Spin (1) and Red_Beyblade_Spin (6) appear
    if hand_launcher_detected and 1 in data and 6 in data and not battle_ongoing:
        hit=0
        battle_ongoing = True
        time_started = True
        winner='none'
        cause='none'
        start_time = time.time()  # Start the timer
        hand_launcher_detected = False  # Reset hand_launcher flag after battle starts
        number_battle += 1  # Increment battle count
        print(f"Battle {number_battle} started: Timer started.")
    
    # check amount of hit each other
    if 3 in data:
        hit+=1

    # Check the winner after both Beyblades are spinning
    if battle_ongoing:
        elapsed_time = time.time() - start_time
        elapsed_millis = elapsed_time * 1000  # Convert to milliseconds
        mins, secs = divmod(elapsed_time, 60)
        millis = int(elapsed_millis % 1000)
        timer_text = f'{int(mins):02}:{int(secs):02}:{millis:03}'

        # If Red_Beyblade_Stop (7) appears before Blue_Beyblade_Stop (2), Blue Beyblade wins
        if 7 in data and not 2 in data:
            battle_ongoing = False
            # print(f"Blue Beyblade wins in Battle {number_battle}!")
            time_started = False
            winner='Blue'
            cause ="Red is stop"
            battle_data=[str(number_battle),str(timer_text),str(winner),str(cause),str(hit)]
            battle_data=['halo']
            save_to_dataframe_csv(battle_data)
            
        # If Blue_Beyblade_Stop (2) appears before Red_Beyblade_Stop (7), Red Beyblade wins
        elif 2 in data and not 7 in data:
            battle_ongoing = False
            # print(f"Red Beyblade wins in Battle {number_battle}!")
            time_started = False
            winner='Red'
            cause = "Blue is stop"
            battle_data=[str(number_battle),str(timer_text),str(winner),str(cause),str(hit)]
            save_to_dataframe_csv(battle_data)
        
        elif is_out_of_arena(bbox):  # Blue Beyblade goes out
                battle_ongoing = False
                cause = 'Red Beyblade is the winner because Blue Beyblade went out of the arena'
                print(f"Red Beyblade wins in Battle {number_battle}!")

        cv2.putText(annotated_frame, f'Timer: {timer_text}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
     #-------------------------------------------------------------------------------------------# 
     
    cv2.putText(annotated_frame, f'The Winner: {winner}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(annotated_frame, f'Fight Number: {hit}', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(annotated_frame, f'Battle No: {number_battle}', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(annotated_frame, f'Caused: {cause}', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Display the frame with annotations
    cv2.imshow('Beyblade Battle Detection', annotated_frame)
    
    # Exit the video if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 Arena, 68.0ms
Speed: 2.3ms preprocess, 68.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Arena, 64.6ms
Speed: 1.0ms preprocess, 64.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Arena, 67.3ms
Speed: 1.9ms preprocess, 67.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Arena, 73.6ms
Speed: 1.6ms preprocess, 73.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Arena, 78.3ms
Speed: 1.6ms preprocess, 78.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Arena, 62.0ms
Speed: 1.9ms preprocess, 62.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Arena, 57.1ms
Speed: 1.6ms preprocess, 57.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Arena, 56.8ms
Speed: 1.6ms preprocess, 56.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


KeyboardInterrupt: 